In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re

#load data
df = pd.read_csv('/content/drive/MyDrive/Assignment 2/Assignment 2.1/Data/train.csv')

# drop id column
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

print("Initial columns:", df.columns.tolist())
print(df.head())


Initial columns: ['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price', 'Price']
                               Name    Location  Year  Kilometers_Driven  \
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015              41000   
1                      Honda Jazz V     Chennai  2011              46000   
2                 Maruti Ertiga VDI     Chennai  2012              87000   
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013              40670   
4            Nissan Micra Diesel XV      Jaipur  2013              86999   

  Fuel_Type Transmission Owner_Type     Mileage   Engine      Power  Seats  \
0    Diesel       Manual      First  19.67 kmpl  1582 CC  126.2 bhp    5.0   
1    Petrol       Manual      First    13 km/kg  1199 CC   88.7 bhp    5.0   
2    Diesel       Manual      First  20.77 kmpl  1248 CC  88.76 bhp    7.0   
3    Diesel    Automatic     Second   15.2 kmpl  1968 CC  14

In [ ]:
df_clean = df.copy()

#keep only numeric vals
unit_cols = ["Mileage", "Engine", "Power", "New_Price"]

def extract_numeric(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    m = re.search(r"(\d+(\.\d+)?)", s)
    return float(m.group(1)) if m else np.nan

for col in unit_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(extract_numeric)

print("\nMissing values before imputation:")
print(df_clean.isna().sum())

# swap nummeric to median
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns

for col in num_cols:
    median_val = df_clean[col].median()
    df_clean[col].fillna(median_val, inplace=True)

# swap categorical to mode
cat_cols = df_clean.select_dtypes(include=["object"]).columns

for col in cat_cols:
    mode_val = df_clean[col].mode()[0]
    df_clean[col].fillna(mode_val, inplace=True)

print("\nMissing values after imputation:")
print(df_clean.isna().sum())



Missing values before imputation:
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

Missing values after imputation:
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


/tmp/ipython-input-1485494217.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(median_val, inplace=True)
/tmp/ipython-input-1485494217.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [ ]:
#one hot encode
cols_to_encode = ["Fuel_Type", "Transmission"]
cols_to_encode = [c for c in cols_to_encode if c in df_clean.columns]

df_encoded = pd.get_dummies(
    df_clean,
    columns=cols_to_encode,
    drop_first=True  # avoids dummy trap
)

print("\nColumns after one-hot encoding:")
print(df_encoded.columns.tolist())



Columns after one-hot encoding:
['Name', 'Location', 'Year', 'Kilometers_Driven', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price', 'Price', 'Fuel_Type_Electric', 'Fuel_Type_Petrol', 'Transmission_Manual']


In [ ]:
#car age for new feature
CURRENT_YEAR = 2025
df_encoded["Car_Age"] = CURRENT_YEAR - df_encoded["Year"]

print("\nCar age preview:")
print(df_encoded[["Year", "Car_Age"]].head())



Car age preview:
   Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


In [ ]:
#select
cols_select = ["Name", "Location", "Year", "Mileage", "Price"]
cols_select = [c for c in cols_select if c in df_encoded.columns]
df_select = df_encoded[cols_select]
print("\nselect")
print(df_select.head())

# filter
if "Mileage" in df_encoded.columns:
    df_filter = df_encoded[df_encoded["Mileage"] > 20]
    print("\nfilter by milage")
    print(df_filter.head())

#rename
rename_map = {}
if "Kilometers_Driven" in df_encoded.columns:
    rename_map["Kilometers_Driven"] = "Odometer_km"

df_renamed = df_encoded.rename(columns=rename_map)
print("\nrename")
print(df_renamed.columns.tolist())

#mutate
df_mutate = df_renamed.copy()
if "Odometer_km" in df_mutate.columns:
    df_mutate["Price_per_km"] = df_mutate["Price"] / df_mutate["Odometer_km"]
    print("\nmutate")
    print(df_mutate[["Price", "Odometer_km", "Price_per_km"]].head())

#arrange
df_arrange = df_mutate.sort_values(by="Price", ascending=True)
print("\narrange")
print(df_arrange[["Name", "Price"]].head())

#summarize and group
group_col = "Transmission_Manual" if "Transmission_Manual" in df_arrange.columns else None

if group_col:
    summary = (
        df_arrange
        .groupby(group_col)
        .agg(
            avg_price=("Price", "mean"),
            avg_mileage=("Mileage", "mean"),
            avg_age=("Car_Age", "mean"),
            count=("Price", "count")
        )
        .reset_index()
    )

    print("\nsummarize and group by")
    print(summary)



select
                               Name    Location  Year  Mileage  Price
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015    19.67  12.50
1                      Honda Jazz V     Chennai  2011    13.00   4.50
2                 Maruti Ertiga VDI     Chennai  2012    20.77   6.00
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013    15.20  17.74
4            Nissan Micra Diesel XV      Jaipur  2013    23.08   3.50

filter by milage
                                  Name Location  Year  Kilometers_Driven  \
2                    Maruti Ertiga VDI  Chennai  2012              87000   
4               Nissan Micra Diesel XV   Jaipur  2013              86999   
6  Volkswagen Vento Diesel Comfortline     Pune  2013              64430   
7       Tata Indica Vista Quadrajet LS  Chennai  2012              65932   
8                     Maruti Ciaz Zeta    Kochi  2018              25692   

  Owner_Type  Mileage  Engine   Power  Seats  New_Price  Price  \
2      First    20.77  1248.0  